In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
from myGenerator import DataGenerator

In [37]:
np.array([[[1,2,3],[4,5,6]],
        [[7,8,9],[10,11,12]],
        [[13,14,15],[16,17,18]]]).shape

(3, 2, 3)

### Carga de datos

In [13]:
train = pd.read_pickle("pan22-authorship-verification-training-dataset/particionesXid/PanTrain.plk")
val = pd.read_pickle("pan22-authorship-verification-training-dataset/particionesXid/PanVal.plk")
test = pd.read_pickle("pan22-authorship-verification-training-dataset/particionesXid/PanTest.plk")
docDict = pd.read_pickle("pan22-authorship-verification-training-dataset/particionesXid/UniquePanDocs.plk")

In [65]:
pair_spectra = train.loc[train.id == '0009e74e-f974-4a4a-a64a-09dff3fbe1e5'].merge(full_spectra, left_on='idtext', right_on='id').spectra

In [74]:
pair_spectra[0].flatten()

array([30.30144882, 18.66531754, 17.78652   , ...,  9.20715427,
        9.08200741,  8.94471073])

In [69]:
pair_spectra[0].flatten().shape

(1200,)

In [51]:
train.columns

Index(['id', 'idtext'], dtype='object')

In [49]:
full_spectra.columns

Index(['id', 'spectra'], dtype='object')

In [54]:
pd.merge(train, full_spectra, left_on='idtext', right_on='id')

,id_x,idtext,id_y,spectra
0,0009e74e-f974-4a4a-a64a-09dff3fbe1e5,377,377,"[[[30.301448822021484, 18.66531753540039, 17.7..."
1,002aa6f2-4980-493d-9bff-34d5bbb6ee23,377,377,"[[[30.301448822021484, 18.66531753540039, 17.7..."
2,0109a2dc-8e31-421c-a470-bf5735a69d55,377,377,"[[[30.301448822021484, 18.66531753540039, 17.7..."
3,01f6ad0c-4cd6-44a2-acf0-f8f8b6637cac,377,377,"[[[30.301448822021484, 18.66531753540039, 17.7..."
4,03c0b1b3-a515-4a96-b86d-8513eb9b54f3,377,377,"[[[30.301448822021484, 18.66531753540039, 17.7..."
...,...,...,...,...
31459,ab8ed5c7-9756-4d21-80de-282cb7cb863b07340,439,439,"[[[31.705324172973633, 18.932188034057617, 17...."
31460,bd154cb1-6f82-4cf4-94f6-449bdae4f38810692,439,439,"[[[31.705324172973633, 18.932188034057617, 17...."
31461,ddfa1db2-b847-431a-b3ad-5663050c306d10493,439,439,"[[[31.705324172973633, 18.932188034057617, 17...."
31462,df6d3c89-473c-48d9-b912-1cc802bfa27811534,439,439,"[[[31.705324172973633, 18.932188034057617, 17...."


In [14]:
def spectraLoader(inputPath: str) -> pd.DataFrame:
    df = pd.DataFrame()
    with open(inputPath, encoding='utf-8') as f:
        for line in f:
            jsonline = json.loads(line)
            lineDf = pd.DataFrame({'id':jsonline['id'], 'spectra':[np.array(jsonline['spectra'])]})
            df = pd.concat([df, lineDf])
    df = df.reset_index(drop=True)

    return df

In [47]:
full_spectra = spectraLoader("pan22-authorship-verification-training-dataset/particionesXid/FullSpectra.jsonl")

In [16]:
def loadLabels(inputPath:str, labelDict:dict) -> dict:
    with open(inputPath, encoding='utf-8') as f:
        for line in f:
            jsonline = json.loads(line)
            labelDict[jsonline['id']] = 1 if jsonline['value'] else 0
        
    return labelDict

In [17]:
labels = dict()
labels = loadLabels('pan22-authorship-verification-training-dataset/particiones/train_truth.jsonl',labels)
labels = loadLabels('pan22-authorship-verification-training-dataset/particiones/val_truth.jsonl',labels)

In [18]:
print("Instances: {}".format((train.shape[0] + val.shape[0])/2))
print("Labels: ",len(labels))

Instances: 16486.0
Labels:  16486


In [19]:
labels["a09fdc6b-ed15-48c5-9d2e-572f989b9b4500000"]

0

In [20]:
partition = {'train': train.id.unique().tolist(), 'validation': val.id.unique().tolist()}

In [21]:
print("Train: ", len(partition['train']))
print("Validation: ", len(partition['validation']))
print("Total: ", len(partition['train'])+len(partition['validation']))

Train:  15732
Validation:  754
Total:  16486


### Definición de la arquitectura de red siamesa

In [9]:
# Modelo de red siamesa utilizando una red residual y un perceptron multicapa

x1 = tf.keras.Input(shape=(1200,), name="Input_1")
x2 = tf.keras.Input(shape=(1200,), name="Input_2")

# Batch normalization
bn_layer = tf.keras.layers.BatchNormalization(name="Batch_normalization")
x1 = bn_layer(x1)
x2 = bn_layer(x2)

# Gaussian noise
gaussian_noise_layer = tf.keras.layers.GaussianNoise(1.0, name="Gaussian_noise")
x1 = gaussian_noise_layer(x1)
x2 = gaussian_noise_layer(x2)

# Dropout
dropout_layer = tf.keras.layers.Dropout(0.4, name="Dropout_1")
x1 = dropout_layer(x1)
x2 = dropout_layer(x2)

# Residual network

substract = lambda x: x[0] - x[1]
residual = tf.keras.layers.Lambda(function=substract, output_shape=lambda x: x[0], name='residual')

dense_layer_1 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_1")
o1_1 = dense_layer_1(x1)
o1_2 = dense_layer_1(x2)

dense_layer_2 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_2")
o2_1 = dense_layer_2(o1_1)
o2_2 = dense_layer_2(o1_2)

# Residual layer
dense_layer_3 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_3")
o3_1 = dense_layer_3(residual([o1_1, o2_1]))
o3_2 = dense_layer_3(residual([o1_2, o2_2]))

dense_layer_4 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_4")
o4_1 = dense_layer_4(o3_1)
o4_2 = dense_layer_4(o3_2)

# Residual layer
dense_layer_5 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_5")
o5_1 = dense_layer_5(residual([o3_1, o4_1]))
o5_2 = dense_layer_5(residual([o3_2, o4_2]))

dense_layer_6 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_6")
o6_1 = dense_layer_6(o5_1)
o6_2 = dense_layer_6(o5_2)

# Residual layer
dense_layer_7 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_7")
o7_1 = dense_layer_7(residual([o5_1, o6_1]))
o7_2 = dense_layer_7(residual([o5_2, o6_2]))

dense_layer_8 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_8")
o8_1 = dense_layer_8(o7_1)
o8_2 = dense_layer_8(o7_2)

# Residual layer
dense_layer_9 = tf.keras.layers.Dense(256, activation='relu', name="ResDense_9")
o9_1 = dense_layer_9(residual([o7_1, o8_1]))
o9_2 = dense_layer_9(residual([o7_2, o8_2]))

# Merging
abs_substract = lambda x: tf.keras.backend.abs(x[0] - x[1])
merge = tf.keras.layers.Lambda(function=abs_substract, output_shape=lambda x: x[0], name='merge')([o9_1,o9_2])

# MLPNN Classifier
hidden = tf.keras.layers.Dense(128, activation='relu', name="hidden_layer")(merge)
dropout_layer2 = tf.keras.layers.Dropout(0.4, name="Dropout_2")(hidden)
output = tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer")(dropout_layer2)

model = tf.keras.Model([x1, x2], output)
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics=["accuracy"])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1200)]       0           []                               
                                                                                                  
 ResDense_1 (Dense)             (None, 256)          307456      ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 ResDense_2 (Dense)             (None, 256)          65792       ['ResDense_1[2][0]',             
                                                                  'ResDense_1[3][0]']             
                                                                                              

In [ ]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)

In [25]:
print("train: ", train.shape[0])
print("test: ", val.shape[0])

train:  31464
test:  1508


In [24]:
print("train: ", train.id.unique().size)
print("test: ", val.id.unique().size)

train:  15732
test:  754


In [ ]:
# Parameters
params = {'dim': (32,32,32),
            'batch_size': 64,
            'n_classes': 6,
            'n_channels': 1,
            'shuffle': True}

In [ ]:
# Datasets
partition = None # IDs
labels = None # Labels

In [ ]:
# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [ ]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)